In [1]:
"""
All background sky scan without prior -> high/low rates

04_get_versatile_background_allsky_scan_high_low_rate.py
"""

'\nAll background sky scan without prior -> high/low rates\n\n04_get_versatile_background_allsky_scan_high_low_rate.py\n'

In [3]:
import sys
import os
import glob
import numpy as np
import pandas as pd
sys.path.append('../../../')
from greco_grb.scripts import SETTING
paths = SETTING.PATH()
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR

from greco_grb.scripts.utils import *


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [4]:
### create arguments arrays ---> input.txt
# 0: grb_name
# 1: batchNtrials
# 2: batchIndex
# 3: tw_in_second
# 4: ncpu
# 5: mode
# 6: outfilename

# Check 1 out of 2

In [19]:
####### Check before running ########
mode = "production"           # production or testing
script_path = '/scratch/cjchen/submit/04.sub'
## [::-1] reverse order
tws_in_second    = [   10,   25,   50,  100,  250,  500][::-1]   
njob_start       = [    0,    0,    0,  100,  100,  100][::-1]
njob_end         = [  100,  100,  100,  100,  100,  100][::-1]
batchsizes       = [ 1000, 1000, 1000, 1000, 1000, 1000][::-1]
grb_name         = "GRB180423A"    # an example grb used to create controled seeds
ncpu             = 4               # multi-core may result in out-of-memory
#######################################

In [20]:
contents = ""
for idx, (tw_in_second, batchNtrials) in enumerate(zip(tws_in_second, batchsizes)):
    for batchIndex in range(njob_start[idx], njob_end[idx]):
        outfilename = "{}_batchSize{}_batchIndex{}_tw{}.npz".format(grb_name, 
                                                                    batchNtrials, 
                                                                    batchIndex, 
                                                                    tw_in_second)
        if os.path.exists(ANA_DIR+"/allsky_scan/no_prior_versatile/high/tw{}/{}".format(tw_in_second, outfilename)) \
        and os.path.exists(ANA_DIR+"/allsky_scan/no_prior_versatile/low/tw{}/{}".format(tw_in_second, outfilename)):
            print("The npz file already exists.")
            print(outfilename)
            continue
            
        contents += "{}\t".format(grb_name)
        contents += "{}\t".format(batchNtrials)
        contents += "{}\t".format(batchIndex)
        contents += "{}\t".format(tw_in_second)
        contents += "{}\t".format(ncpu)
        contents += "{}\t".format(mode)
        contents += "{}\n".format(outfilename)

with open("04.txt", 'w') as writer:
    writer.write(contents)

In [21]:
print(contents)

GRB180423A	1000	0	50	4	production	GRB180423A_batchSize1000_batchIndex0_tw50.npz
GRB180423A	1000	1	50	4	production	GRB180423A_batchSize1000_batchIndex1_tw50.npz
GRB180423A	1000	2	50	4	production	GRB180423A_batchSize1000_batchIndex2_tw50.npz
GRB180423A	1000	3	50	4	production	GRB180423A_batchSize1000_batchIndex3_tw50.npz
GRB180423A	1000	4	50	4	production	GRB180423A_batchSize1000_batchIndex4_tw50.npz
GRB180423A	1000	5	50	4	production	GRB180423A_batchSize1000_batchIndex5_tw50.npz
GRB180423A	1000	6	50	4	production	GRB180423A_batchSize1000_batchIndex6_tw50.npz
GRB180423A	1000	7	50	4	production	GRB180423A_batchSize1000_batchIndex7_tw50.npz
GRB180423A	1000	8	50	4	production	GRB180423A_batchSize1000_batchIndex8_tw50.npz
GRB180423A	1000	9	50	4	production	GRB180423A_batchSize1000_batchIndex9_tw50.npz
GRB180423A	1000	10	50	4	production	GRB180423A_batchSize1000_batchIndex10_tw50.npz
GRB180423A	1000	11	50	4	production	GRB180423A_batchSize1000_batchIndex11_tw50.npz
GRB180423A	1000	12	50	4	production	G

In [22]:
print("Total number of jobs: {}\n".format(contents.count('\n')))

Total number of jobs: 300



In [23]:
### create pbs file ---> 04.pbs

# Check 2 out of 2

In [25]:
nameofjob  = "04"
ntotal     = 300             # copy from the above 2nd cell "Total number of jobs: xxx"
inputtxt   = "04.txt"      # copy from the above 3rd cell
outdir     = ensure_dir("/storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out")
walltimehr = 60           # in Hour: 1 cpu min / trial (x2 for redundancy) < 120 hrs

In [26]:
contents = ""
contents += "#PBS -N {}\n".format(nameofjob)
contents += "#PBS -l nodes=1:ppn={}\n".format(ncpu)           # number of nodes and cores per node required
contents += "#PBS -l pmem=4gb\n"                             # memory per core
contents += "#PBS -l walltime={}:00:00\n".format(walltimehr)  # dd:hh:mm:ss
contents += "#PBS -q hive\n"
contents += "#PBS -j oe\n"
contents += "#PBS -o {}\n".format(outdir+"/04.out")
contents += "#PBS -t 1-{}%400".format(ntotal)
contents += "\n"
contents += "cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts\n"
contents += "icpy3\n" 
# Get Paramters from input.txt file using $PBS_ARRAYID as the line number 
contents += 'params=`sed -n "${{PBS_ARRAYID}} p" {}`\n'.format("hive_sub/"+inputtxt)    # outer single quote to preserve inner double quotes
contents += "paramsArray=($params)\n"
contents += "arg0=${paramsArray[0]}\n"
contents += "arg1=${paramsArray[1]}\n"
contents += "arg2=${paramsArray[2]}\n"
contents += "arg3=${paramsArray[3]}\n"
contents += "arg4=${paramsArray[4]}\n"
contents += "arg5=${paramsArray[5]}\n"
contents += "arg6=${paramsArray[6]}\n"
contents += "python 04_get_versatile_background_allsky_scan_high_low_rate.py "
contents += "--grb_name $arg0 "
contents += "--batchNtrials $arg1 "
contents += "--batchIndex $arg2 "
contents += "--tw_in_second $arg3 "
contents += "--ncpu $arg4 "
contents += "--mode $arg5 "
contents += "--outfilename $arg6 "
print(contents)
with open("04.pbs", 'w') as writer:
    writer.write(contents)

#PBS -N 04
#PBS -l nodes=1:ppn=4
#PBS -l pmem=4gb
#PBS -l walltime=60:00:00
#PBS -q hive
#PBS -j oe
#PBS -o /storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out/04.out
#PBS -t 1-300%400
cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts
icpy3
params=`sed -n "${PBS_ARRAYID} p" hive_sub/04.txt`
paramsArray=($params)
arg0=${paramsArray[0]}
arg1=${paramsArray[1]}
arg2=${paramsArray[2]}
arg3=${paramsArray[3]}
arg4=${paramsArray[4]}
arg5=${paramsArray[5]}
arg6=${paramsArray[6]}
python 04_get_versatile_background_allsky_scan_high_low_rate.py --grb_name $arg0 --batchNtrials $arg1 --batchIndex $arg2 --tw_in_second $arg3 --ncpu $arg4 --mode $arg5 --outfilename $arg6 
